<a href="https://colab.research.google.com/github/PascalPolygon/AffineCNN/blob/main/AffineCNN_experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import torch
import torchvision
import matplotlib.pyplot as plt
from time import time
from torchvision import datasets, transforms
from torch import nn, optim
from google.colab import drive
import matplotlib.pyplot as plt
import cv2
from google.colab.patches import cv2_imshow
import imutils 
from PIL import Image,  ImageOps
from matplotlib import cm
from scipy import ndimage
import torchvision.transforms.functional as TF
import random
import os
from glob import glob
from torch.utils.data import Dataset, DataLoader
from torchvision.io import read_image

In [40]:
drive.mount('/content/drive', force_remount=True)
MNIST_TRAIN = '/content/drive/MyDrive/translation_vector_estimation_project/MNIST_TRAIN'
MNIST_TEST = '/content/drive/MyDrive/translation_vector_estimation_project/MNIST_TEST'
# MNIST_ROTATED_IMAGES = 'content/drive/MyDrive/translation_vector_estimation_project/MNIST_ROTATED/images'
MNIST_ROTATED_IMAGES = '/content/drive/MyDrive/translation_vector_estimation_project/MNIST_ROTATED/images'
MNIST_ROTATED_LABELS = '/content/drive/MyDrive/translation_vector_estimation_project/MNIST_ROTATED/labels'
MODELS_DIR = '/content/drive/MyDrive/translation_vector_estimation_project/MNIST_ROTATED/models'

Mounted at /content/drive


In [159]:
# transform = transforms.Compose([transforms.RandomRotation(45),
#                               transforms.ToTensor(),
#                               transforms.Normalize((0.5,), (0.5,)),
#                               ])
# transform=transforms.Compose([
#     transforms.ToTensor(),
#      transforms.Normalize((0.5,), (0.5,))
# ])

In [42]:
transform=transforms.Compose([
    transforms.ToTensor(),
     transforms.Normalize((0.5,), (0.5,))
])
trainset = datasets.MNIST(MNIST_TRAIN, download=True, train=True, transform=transform)
valset = datasets.MNIST(MNIST_TEST, download=True, train=False, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True)
valloader = torch.utils.data.DataLoader(valset, batch_size=32, shuffle=True)

In [ ]:
# dataiter = iter(trainloader)
# images, labels = dataiter.next()
# for i, data in enumerate(trainloader):
#   image, label = data
#   genRotated(image, )
#   print(label)

for i, data in enumerate(trainloader):
  # angle = random.randint(-45, 45)
  im, label = data
  angle = random.uniform(-45, 45)
  im = TF.rotate(im, angle)
  print(angle)

  rot_path = os.path.join(MNIST_ROTATED_IMAGES, f'{i}.png')
  label_path = os.path.join(MNIST_ROTATED_LABELS, f'{i}.npy')
  print(im.shape)
  im_T = np.transpose(im[0], (1, 2, 0))
  print(im_T.shape)
  x = cv2.cvtColor(np.float32(im_T),cv2.COLOR_GRAY2RGB)
  # print(backtorgb.shape)
  x = (x-np.amin(x))/(np.amax(x)-np.amin(x))
  # print(backtorgb[:,:,0])
  plt.imsave(rot_path, x, cmap='gray')
  np.save(label_path, np.array(angle))
  # plt.imshow(im[0], cmap='gray')
print(images.shape)
print(labels.shape)

41.805235723568586
torch.Size([32, 1, 28, 28])
torch.Size([28, 28, 1])
-18.294810705351185
torch.Size([32, 1, 28, 28])
torch.Size([28, 28, 1])
24.513963266406535
torch.Size([32, 1, 28, 28])
torch.Size([28, 28, 1])
24.63760895372208
torch.Size([32, 1, 28, 28])
torch.Size([28, 28, 1])
-0.8398555835105128
torch.Size([32, 1, 28, 28])
torch.Size([28, 28, 1])
16.600731386273665
torch.Size([32, 1, 28, 28])
torch.Size([28, 28, 1])
-24.857234658173986
torch.Size([32, 1, 28, 28])
torch.Size([28, 28, 1])
4.3243753476969005
torch.Size([32, 1, 28, 28])
torch.Size([28, 28, 1])
-4.8474031452325335
torch.Size([32, 1, 28, 28])
torch.Size([28, 28, 1])
23.14139821222632
torch.Size([32, 1, 28, 28])
torch.Size([28, 28, 1])
14.91729890353303
torch.Size([32, 1, 28, 28])
torch.Size([28, 28, 1])
-13.876470485691435
torch.Size([32, 1, 28, 28])
torch.Size([28, 28, 1])
-20.02244759903462
torch.Size([32, 1, 28, 28])
torch.Size([28, 28, 1])
2.710763813953342
torch.Size([32, 1, 28, 28])
torch.Size([28, 28, 1])
-26.7

In [36]:
class RotatedMNISTDataset(Dataset):
  def __init__(self, img_dir, labels_dir, transform=None, target_transform=None):
        self.labels_dir = labels_dir
        self.img_dir = img_dir
        self.paths = []
        for path in glob(self.img_dir+'/*png'):
          self.paths.append(path)
        self.transform = transform
        self.target_transform = target_transform

  def __len__(self):
      return len(self.paths)

  def __getitem__(self, idx):
      # img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
      # img_path = os.path.join(self.img_dir, f'{idx}.png')
      img_path = self.paths[idx]
      filename = os.path.basename(img_path).split('.')[0]
      label_path = os.path.join(self.labels_dir, f'{filename}.npy')
      image = read_image(img_path)[:3,:,:] 
      label = np.load(label_path)
      if self.transform:
          image = self.transform(image)
      return image, label

transform = transforms.Compose([
                    transforms.ToPILImage(),
                    transforms.Grayscale(1),
                    transforms.ToTensor(),
])

dataset = RotatedMNISTDataset(MNIST_ROTATED_IMAGES, MNIST_ROTATED_LABELS, transform=transform)
# dataset = RotatedMNISTDataset(MNIST_ROTATED_IMAGES, MNIST_ROTATED_LABELS, transform=None)
print(len(dataset))

trainloader = DataLoader(dataset, batch_size=32, num_workers=2, shuffle=True)

22698


In [37]:
class RotNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Conv2d(1, 8, kernel_size=3, padding='same'),
            nn.BatchNorm2d(8),
            nn.ReLU(inplace=True),
            nn.AvgPool2d(2, 2),

            nn.Conv2d(8, 16, kernel_size=3, padding='same'),
            nn.BatchNorm2d(16),
            nn.ReLU(inplace=True),
            nn.AvgPool2d(2, 2),

            nn.Conv2d(16, 32, kernel_size=3, padding='same'),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),

            nn.Conv2d(32, 32, kernel_size=3, padding='same'),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True)
        )

        self.classifier = nn.Sequential(
            nn.Dropout(p=0.2),
            nn.Linear(1568, 1)
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
      x = self.layers(x)
      x = torch.flatten(x, 1)
      x = self.classifier(x)
      return x


net = RotNet()

if torch.cuda.is_available():
  print(f'GPU : {torch.cuda.get_device_name(0)}')
  # cuda0 = torch.device('cuda:0')
  device = torch.device("cuda:0")
  net.cuda()
else:
  device = "cpu"

print('---------- Network initialized -------------')
num_params = 0
for param in net.parameters():
  num_params += param.numel()
print('[Network] Total number of parameters : %.3f M' % (num_params / 1e6))
print('-----------------------------------------------')

GPU : Tesla P100-PCIE-16GB
---------- Network initialized -------------
[Network] Total number of parameters : 0.017 M
-----------------------------------------------


In [38]:
criterion = nn.MSELoss()
# optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
optimizer = optim.Adam(net.parameters(), lr=0.0001, betas=(0.9, 0.999))
# print(net.layers)
# print(net.classifier)

# Training

In [39]:
iterations = []
losses = []
myI = 0
for epoch in range(30):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        inputs, labels = inputs.to(device), labels.to(device)

        inputs = inputs.type(torch.cuda.FloatTensor)
        labels = labels.type(torch.cuda.FloatTensor)
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        myI += 1
        if i % 10 == 9:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 10))
            # torch.save(model, os.path.join(MODELS_DIR, f'm_{epoch}_{running_loss/10}.pth'))
            PATH = os.path.join(MODELS_DIR, f'm_{epoch}_{running_loss/10}.pth')
            torch.save(net.state_dict(), PATH)
            running_loss = 0.0
            iterations.append(myI)
            losses.append(running_loss/10)
      

print('Finished Training')

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:520: UserWarning: Using a target size (torch.Size([32])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


FileNotFoundError: ignored

In [ ]:
plt.plot(iterations, losses)
plt.xlabel('iter')
plt.ylabel('MSE loss')
plt.show()